# ⚙️ Utilisation des outils avec les modèles LLM

## 🎯 Pourquoi utiliser des outils avec un LLM ?

Sans outils, un LLM ne fait qu’**interpréter et générer du texte**. Cela limite ses capacités à :

- manipuler des données dynamiques,
- interagir avec des systèmes réels,
- ou répondre à des questions nécessitant une logique métier.

En intégrant des outils, vous offrez au LLM la possibilité de **devenir un véritable agent intelligent** capable de :

✅ prendre des décisions,  
✅ exécuter des actions dans le monde réel,  
✅ et **travailler avec précision sur des données vivantes**.

## 📌 Qu’est-ce qu’un outil (Tool) dans le contexte des LLM ?

Un **outil** est une fonction ou une API externe que le modèle de langage peut invoquer pour accomplir une tâche spécifique qu’il **ne peut pas exécuter seul**, comme :

- accéder à une base de données,
- effectuer un calcul complexe,
- interroger une API tierce,
- exécuter du code,
- consulter des documents internes,
- ou encore lancer une recherche sur Internet.

Les outils permettent d’**étendre les capacités** du modèle en le connectant à des fonctions exécutables dans un environnement réel.


## ⚙️ Création des Tools avec `@tool`

LangChain propose le décorateur `@tool` pour **simplifier la création d’outils**. Il offre plusieurs avantages importants :

### ✅ Déduction automatique
Le décorateur :
- **déduit automatiquement** le nom de l’outil,
- génère une **description compréhensible par le modèle**,
- identifie les **paramètres attendus** selon la signature de la fonction.

### 🧪 Personnalisation
Il est aussi possible de :
- personnaliser le nom et la description de l’outil,
- restreindre ou masquer certains paramètres à l’intention du modèle.

### 🖼️ Outils retournant des artefacts
Les outils peuvent aussi retourner des objets non textuels appelés **artefacts**, comme :
- des **images**,
- des **dataframes** (Pandas),
- ou tout autre type de données utiles.

### 🙈 Arguments injectés
Il est possible d’**injecter des arguments** directement dans la fonction, sans que ceux-ci n’apparaissent dans le schéma visible du modèle. Cela permet de **masquer des paramètres sensibles ou techniques** tout en les utilisant en arrière-plan.


In [ ]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
   """Fais la multiplication."""
   return a * b

Vous pouvez également inspecter le schéma de l'outil et d'autres propriétés :

```python
print(multiply.name) # multiply
print(multiply.description) # Multiply two numbers.
print(multiply.args) 
# {
# 'type': 'object', 
# 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 
# 'required': ['a', 'b']
# }

### Injection des paramètres

Dans certains cas, certains arguments doivent être transmis à un outil au moment de l'exécution, mais ne doivent pas être générés par le modèle lui-même. Pour cela, nous utilisons l'annotation InjectedToolArg, qui permet de masquer certains paramètres dans le schéma de l'outil.

Par exemple, si un outil nécessite qu'un user_id soit injecté dynamiquement au moment de l'exécution, il peut être structuré de cette manière :

In [ ]:
from langchain_core.tools import tool, InjectedToolArg

@tool
def user_specific_tool(input_data: str, user_id: InjectedToolArg) -> str:
    """Tool that processes input data."""
    return f"User {user_id} processed {input_data}"

### Artéfacts de l'outil

Les outils sont des utilitaires qui peuvent être appelés par un modèle et dont les résultats sont conçus pour être renvoyés à un modèle. Parfois, cependant, il existe des artefacts de l'exécution d'un outil que nous voulons rendre accessibles aux composants en aval de notre chaîne ou agent, mais que nous ne voulons pas exposer au modèle lui-même. Par exemple, si un outil renvoie un objet personnalisé, un cadre de données ou une image, nous pouvons vouloir transmettre des métadonnées sur cette sortie au modèle sans lui transmettre la sortie elle-même. En même temps, nous pouvons vouloir accéder à ce résultat complet ailleurs, par exemple dans des outils en aval.